# Preparation of a specific dataset file for Elemental descriptor

This part of the code is reading only A-site, B-site, and Chemical formula information from perovskites.xlsx file and generates a table of compositions for the compounds in the dataset.

The header of the output file (named Johns_database.txt) should be end up:

An example of the feature vector for the (Bi100.0)(Ca100.0)(Fe100.0)()O3 double-perovskite compound (please notice that the information about oxygen is not used in this version):

Python was installed using conda. A couple of bash commands to activate the environemnt:

In [29]:
#Importing packages
import pandas as pd
import chemparse
import os
import platform
import sys

In [10]:
print(f"Python Platform: {platform.platform()}")
print(f"Pandas {pd.__version__}")
print(f"Python {sys.version}")

Python Platform: macOS-12.6-arm64-arm-64bit
Pandas 1.4.2
Python 3.9.15 | packaged by conda-forge | (main, Nov 22 2022, 08:48:25) 
[Clang 14.0.6 ]


In [30]:
#!/usr/bin/python3
'''
Created on Nov 26, 2022

@authors: Jiri Hostas & Maicon Lourenco
Last modification by JH: 10/01/2023
'''

def reading_file(file_name):
    global elements_A1_site_list, elements_A2_site_list, elements_B1_site_list, elements_B2_site_list
    
    df = pd.read_excel(file_name)
    
    cleaned = pd.DataFrame()
    cleaned['A-site']=df['A-site'].str.replace(" ","").str.replace(" ","")
    cleaned['B-site']=df['B-site'].str.replace(" ","").str.replace(" ","")
    cleaned['Chemical Formula']=df['Chemical Formula']
    cleaned['Chemical Formula Dictionary']=df['Chemical Formula']

    cleaned[['A1-site','A2-site']] = df['A-site'].str.split(",",expand=True,)
    cleaned[['B1-site','B2-site']] = df['B-site'].str.split(",",expand=True,)

    cleaned['A1-site']=cleaned['A1-site'].str.replace(" ","")
    cleaned['A2-site']=cleaned['A2-site'].str.replace(" ","")
    cleaned['B1-site']=cleaned['B1-site'].str.replace(" ","")
    cleaned['B2-site']=cleaned['B2-site'].str.replace(" ","")
    cleaned = cleaned.drop(columns='A-site').drop(columns='B-site').fillna('Empty')
    
    lenght=len(cleaned['Chemical Formula'])

    for row in range(lenght):
        cleaned['Chemical Formula Dictionary'][row]= chemparse.parse_formula(cleaned['Chemical Formula'][row])
    
    matrix = cleaned['Chemical Formula Dictionary'].apply(pd.Series)
    matrix['sum'] = matrix.sum(axis = 1)
    wlenght=len(matrix.columns)

    for col in range(wlenght):
        matrix.iloc[:, col] = matrix.iloc[:, col] *100 #/ matrix['sum']
        
    compositions = matrix.drop(columns='sum').fillna(0)
    
    lenght=len(cleaned['A1-site']) # 85 for current dataset


    for row in range(lenght):
        cleaned['A1-site'][row]= chemparse.parse_formula(cleaned['A1-site'][row])
        cleaned['A2-site'][row]= chemparse.parse_formula(cleaned['A2-site'][row])
        cleaned['B1-site'][row]= chemparse.parse_formula(cleaned['B1-site'][row])
        cleaned['B2-site'][row]= chemparse.parse_formula(cleaned['B2-site'][row])
    
    # Matrices telling us if there is A1/A2/B1/B2 element in the compound (values: 1 or 0)
    matrixA1 = cleaned['A1-site'].apply(pd.Series)
    matrixA2 = cleaned['A2-site'].apply(pd.Series)
    matrixB1 = cleaned['B1-site'].apply(pd.Series)
    matrixB2 = cleaned['B2-site'].apply(pd.Series)

    # Combining the existence of element and chemical formula information. 
    resultsA1 = (matrixA1*compositions).fillna(0)
    resultsA1 = resultsA1.loc[:, (resultsA1 != 0).any(axis=0)]
    resultsA2 = (matrixA2*compositions).fillna(0).drop(columns='Empty')
    resultsA2= resultsA2.loc[:, (resultsA2 != 0).any(axis=0)]
    resultsB1 = (matrixB1*compositions).fillna(0)
    resultsB1= resultsB1.loc[:, (resultsB1 != 0).any(axis=0)]
    resultsB2 = (matrixB2*compositions).fillna(0).drop(columns='Empty')
    resultsB2= resultsB2.loc[:, (resultsB2 != 0).any(axis=0)]
    
    # Reading the Y value, in this case I have used only sample preparation 0=unknown, 1=known.
    # This is not used for inference but mostly as a placeholder.
    df['Sample preparation'].replace(to_replace='unknown', value="1", regex=True, inplace=True)
    df['Sample preparation'].replace(to_replace='[^0-9]+', value='0', regex=True, inplace=True)
    resultsY = df['Sample preparation']
    
    print("A1 array is: "), print(resultsA1.columns)
    elements_A1_site_list = resultsA1.columns
    print("A2 array is: "), print(resultsA2.columns)
    elements_A2_site_list = resultsA2.columns
    print("B1 array is: "), print(resultsB1.columns)
    elements_B1_site_list = resultsB1.columns
    print("B2 array is: "), print(resultsB2.columns)
    elements_B2_site_list = resultsB2.columns
    
    resultsA1.to_csv(path+'resultsA1'+'.csv', index=False)
    resultsA2.to_csv(path+'resultsA2'+'.csv', index=False)
    resultsB1.to_csv(path+'resultsB1'+'.csv', index=False)
    resultsB2.to_csv(path+'resultsB2'+'.csv', index=False)
    resultsY.to_csv(path+'resultsY'+'.csv', index=False)

    # I could rewrite this using subprocess package in the future
    # https://docs.python.org/3.9/library/subprocess.html
    # pay attention to change the destinations
    # !mkdir /Users/hostas/Documents/01-Analysis/2022-11-24_Maicon_Perovskites/database 2>/dev/null
    # !paste /Users/hostas/Documents/01-Analysis/2022-11-24_Maicon_Perovskites/database/results* > /Users/hostas/Documents/01-Analysis/2022-11-24_Maicon_Perovskites/database/Johns_database.csv ; rm /Users/hostas/Documents/01-Analysis/2022-11-24_Maicon_Perovskites/database/results*
    # !sed 's/,/   /g' /Users/hostas/Documents/01-Analysis/2022-11-24_Maicon_Perovskites/database/Johns_database.csv | sed 's/Sample preparation/Sample_Preparation/g' > /Users/hostas/Documents/01-Analysis/2022-11-24_Maicon_Perovskites/database/Johns_database.txt


path = '/Users/hostas/Documents/01-Analysis/2022-11-24_Maicon_Perovskites/database/'
#location of the original xlsx file:
file_name=r'/Users/hostas/Documents/01-Analysis/2022-11-06 Perovskites/perovskites.xlsx'
reading_file(file_name)

A1 array is: 
Index(['Ba', 'Bi', 'Ca', 'Dy', 'Eu', 'Gd', 'La', 'Li', 'Lu', 'Na', 'Pr', 'Sm',
       'Sr', 'Tb', 'Y'],
      dtype='object')
A2 array is: 
Index(['Ba', 'Ca', 'Cu', 'Eu', 'Gd', 'Mg', 'Na', 'Sm', 'Sr'], dtype='object')
B1 array is: 
Index(['Ag', 'Al', 'Bi', 'Ce', 'Co', 'Cr', 'Dy', 'Eu', 'Fe', 'Ga', 'Gd', 'In',
       'La', 'Mg', 'Mn', 'Nd', 'Ni', 'Pr', 'Sc', 'Sm', 'Ta', 'Ti', 'Zn'],
      dtype='object')
B2 array is: 
Index(['Bi', 'Cr', 'Fe', 'I', 'Mn', 'Mo', 'Nb', 'Ta', 'Te', 'Ti', 'V', 'W'], dtype='object')


# Elemental Descriptor

In [21]:
#!/usr/bin/python3
'''
Created on Novemeber 26, 2022

@author: Maicon Lourenco & Jiri Hostas
Last modification by JH: 28/11/2022.
'''

import pymatgen.core as mg
import math
import sys

# https://pymatgen.org/pymatgen.core.bonds.html
def get_properties(is_debug, A_site_elements, A_site_compositions,
                             B_site_elements, B_site_compositions):

    X_A = {} # weighted properties of elements in A
    X_B = {} # weighted properties of elements in B

    obj_A = None
    obj_B = None

    # A
    r_A_list = []
    ideal_bond_length_A_O_list = []
    electronegativity_A_list = []
    van_der_walls_radius_A_list = []
    ionization_energy_A_list = []
    molar_volume_A_list = []
    atomic_number_A_list = []
    atomic_mass_A_list = []

    # B
    r_B_list = []
    ideal_bond_length_B_O_list = []
    electronegativity_B_list = []
    van_der_walls_radius_B_list = []
    ionization_energy_B_list = []
    molar_volume_B_list = []
    atomic_number_B_list = []
    atomic_mass_B_list = []

    properties_A = {}
    properties_B = {}

    if is_debug == True:
       print("\n-------")
        

    for i in range(len(A_site_elements)):

        obj_A = mg.Species(A_site_elements[i], +2)

        # Shannon's (1976) ionic radii of A-site (12-coordination)
        r_A = mg.Element(A_site_elements[i]).average_cationic_radius

        #r_A = (obj_A.get_shannon_radius(cn="XII", spin="Low Spin", radius_type="ionic")).real
        #r_A_list.append(r_A)

        #r_A = (obj_A.get_shannon_radius(cn="XII", spin="Low Spin", radius_type="ionic")).real
        r_A_list.append(r_A.real)
        properties_A["r_A"] = r_A_list

        if is_debug == True:
           print("\nShannon's ionic raddi A:", r_A, A_site_elements[i], A_site_compositions[i])

        # A-O bond distance from the sum of atomic_radius
        ideal_bond_length_A_O = (mg.Element(A_site_elements[i]).atomic_radius + mg.Element("O").atomic_radius).real
        ideal_bond_length_A_O_list.append(ideal_bond_length_A_O)
        properties_A["ideal_bond_length_A_O"] = ideal_bond_length_A_O_list

        if is_debug == True:
           print("ideal_bond_lenght_A_O:", ideal_bond_length_A_O, A_site_elements[i], A_site_compositions[i])

        # A site electronegativity
        electronegativity_A = mg.Element(A_site_elements[i]).X
        electronegativity_A_list.append(electronegativity_A)
        properties_A["electronegativity_A"] = electronegativity_A_list

        if is_debug == True:
           print("electronegativity A:", electronegativity_A, A_site_elements[i], A_site_compositions[i])

        # A site van der Waals radius
        van_der_walls_radius_A = mg.Element(A_site_elements[i]).van_der_waals_radius.real
        van_der_walls_radius_A_list.append(van_der_walls_radius_A)
        properties_A["van_der_walls_radius_A"] = van_der_walls_radius_A_list

        if is_debug == True:
            print("van_der_waals_radius A:", van_der_walls_radius_A, A_site_elements[i], A_site_compositions[i])

        # A site ionization energy
        ionization_energy_A = mg.Element(A_site_elements[i]).ionization_energies[0]
        #electronegativity_A_list.append(ionization_energy_A) # it was a bug
        ionization_energy_A_list.append(ionization_energy_A) # it fixed the bug
        #properties_A["ionization_energy_A"] = electronegativity_A_list
        properties_A["ionization_energy_A"] = ionization_energy_A_list

        if is_debug == True:
           print("ionization_energy A:", ionization_energy_A, A_site_elements[i], A_site_compositions[i])

        # A molar volume in cm^3
        molar_volume_A = mg.Element(A_site_elements[i]).molar_volume.real
        molar_volume_A_list.append(molar_volume_A)
        properties_A["molar_volume_A"] = molar_volume_A_list

        if is_debug == True:
           print("molar_volume A:", molar_volume_A, A_site_elements[i], A_site_compositions[i])

        # A atomic number
        atomic_number_A = mg.Element(A_site_elements[i]).Z
        atomic_number_A_list.append(atomic_number_A)
        properties_A["atomic_number_A"] = atomic_number_A_list

        if is_debug == True:
           print("atomic_number A:", atomic_number_A, A_site_elements[i], A_site_compositions[i])

        # A atomic mass amu
        atomic_mass_A = mg.Element(A_site_elements[i]).atomic_mass.real
        atomic_mass_A_list.append(atomic_mass_A)
        properties_A["atomic_mass_A"] = atomic_mass_A_list

        if is_debug == True:
           print("atomic_mass A:", atomic_mass_A, A_site_elements[i], A_site_compositions[i])

    #print("size of properties A:", len(properties_A))

    for i in range(len(B_site_elements)):

        obj_B = mg.Species(B_site_elements[i], +4)

        r_B = mg.Element(B_site_elements[i]).average_cationic_radius

        r_B_list.append(r_B.real)
        properties_B["r_B"] = r_B_list

        if is_debug == True:
            print("\nShannon's ionic raddi B:", r_B, B_site_elements[i], B_site_compositions[i])

        # A-O bond distance from the sum of atomic_radius
        ideal_bond_length_B_O = (mg.Element(B_site_elements[i]).atomic_radius + mg.Element("O").atomic_radius).real
        ideal_bond_length_B_O_list.append(ideal_bond_length_B_O)
        properties_B["ideal_bond_length_B_O"] = ideal_bond_length_B_O_list

        if is_debug == True:
           print("ideal_bond_lenght_B_O:", ideal_bond_length_B_O, B_site_elements[i], B_site_compositions[i])

        # B site electronegativity
        electronegativity_B = mg.Element(B_site_elements[i]).X.real
        electronegativity_B_list.append(electronegativity_B)
        properties_B["electronegativity_B"] = electronegativity_B_list

        if is_debug == True:
           print("electronegativity B:", electronegativity_B, B_site_elements[i], B_site_compositions[i])

        # B site van der Waals radius
        van_der_walls_radius_B = mg.Element(B_site_elements[i]).van_der_waals_radius.real
        van_der_walls_radius_B_list.append(van_der_walls_radius_B)
        properties_B["van_der_walls_radius_B"] = van_der_walls_radius_B_list

        if is_debug == True:
           print("van_der_waals_radius B:", van_der_walls_radius_B, B_site_elements[i], B_site_compositions[i])

        # B site ionization energy
        ionization_energy_B = mg.Element(B_site_elements[i]).ionization_energies[0]
        ionization_energy_B_list.append(ionization_energy_B)
        properties_B["ionization_energy_B"] = ionization_energy_B_list

        if is_debug == True:
           print("ionization_energy B:", ionization_energy_B, B_site_compositions[i])

        # B molar volume in cm^3
        molar_volume_B = mg.Element(B_site_elements[i]).molar_volume.real
        molar_volume_B_list.append(molar_volume_B)
        properties_B["molar_volume_B"] = molar_volume_B_list

        if is_debug == True:
           print("molar_volume B:", molar_volume_B, B_site_elements[i], B_site_compositions[i])

        # B atomic number
        atomic_number_B = mg.Element(B_site_elements[i]).Z
        atomic_number_B_list.append(atomic_number_B)
        properties_B["atomic_number_B"] = atomic_number_B_list

        if is_debug == True:
           print("atomic_number B:", atomic_number_B, B_site_elements[i], B_site_compositions[i])

        # B atomic mass amu
        atomic_mass_B = mg.Element(B_site_elements[i]).atomic_mass.real
        atomic_mass_B_list.append(atomic_mass_B)
        properties_B["atomic_mass_B"] = atomic_mass_B_list

        if is_debug == True:
           print("atomic_mass B:", atomic_mass_B, B_site_elements[i], B_site_compositions[i])

    #print("size of properties B:", len(properties_B))    
    # ------------------------------------------ #
    # BEGIN getting averaged properties: X_A/X_B #
    # ------------------------------------------ #

    # XA
    tmp01_A = 0.0
    for iProperties in properties_A: # n_properties
        tmp01_A = 0.0
        for j in range(len(A_site_compositions)):

             #A_site_elements, A_site_compositions
             #print("MMA01:", j, iProperties, properties_A[iProperties][j], A_site_compositions[j], A_site_elements[j], len(A_site_compositions))
             tmp01_A = tmp01_A + \
             (A_site_compositions[j]*properties_A[iProperties][j])

        X_A[iProperties] = tmp01_A
        #print("MMA02:", iProperties, tmp01_A, X_A[iProperties])

    # XB
    tmp01_B = 0.0
    for iProperties in properties_B: # n_properties
        tmp01_B = 0.0
        for j in range(len(B_site_compositions)):

             #B_site_elements, B_site_compositions
             #print("MMB01:", j, iProperties, properties_B[iProperties][j], B_site_compositions[j], B_site_elements[j], len(B_site_compositions))
             tmp01_B = tmp01_B + \
             (B_site_compositions[j]*properties_B[iProperties][j])

        X_B[iProperties] = tmp01_B
        #print("MMB02:", iProperties, tmp01_B, X_B[iProperties])

    if is_debug == True:
       print("----------------------")
       print("properties_A:", properties_A)
       print("properties_B:", properties_B)
       print("average properties, X_A:", X_A)
       print("average properties, X_B:", X_B)

    return properties_A, X_A, properties_B, X_B

    # --------------------------------------- #
    # END getting averaged properties: X_A/XB #
    # --------------------------------------- #

def get_initial_data(is_debug, file_name):

    X_init = []
    y_init = []
    elements_list = []

    file_in = open(file_name, "r")

    file_lines = file_in.readlines()

    file_in.close()
    tmp_list = []

    for i in range(len(file_lines)):

        tmp_lines = file_lines[i].split()
        tmp_list = []

        for j in range(0, len(tmp_lines) - 1):

            if i == 0:
               tmp_list.append(tmp_lines[j])
            else:
               tmp_list.append(float(tmp_lines[j]))

        if i > 0:

           X_init.append(tmp_list)
           y_init.append(tmp_lines[-1])

        tmp_lines = []
        tmp_list = []

    if is_debug == True:
       for i in range(len(X_init)):
           print ("X and y:", X_init[i], y_init[i])

    return X_init, y_init, elements_list

def main(is_debug, is_print_unit_formula, 
         elements_A1_site_list, elements_A2_site_list,
         elements_B1_site_list, elements_B2_site_list,
         file_name):

    X_init, y_init, elements_list = get_initial_data(is_debug, file_name)

    total_elements = len(elements_A1_site_list + elements_A2_site_list + elements_B1_site_list + elements_B2_site_list)

    # error handling
    try:
        if len(X_init[0]) != total_elements:
           raise
    except:
        print("\nError!")
        print("The number of elements in 'A_site_list' + 'B_site_list' is: " + str(total_elements))
        print("The number of elements (columns) in '" + file_name + "' is: " + str(len(X_init[0])))
        print("They must be equal.\n")
        sys.exit()

    A1_site_indexes = [i for i in range(len(elements_A1_site_list))]
    A2_site_indexes = [i + (len(elements_A1_site_list)) for i in range(len(elements_A2_site_list))]
    B1_site_indexes = [i + 1 + A2_site_indexes[-1] for i in range(len(elements_B1_site_list))]
    B2_site_indexes = [i  + 1 + B1_site_indexes[-1] for i in range(len(elements_B2_site_list))]
    #B1_site_indexes = [i + (len(elements_A2_site_list)) for i in range(len(elements_B1_site_list))]
    #B2_site_indexes = [i + (len(elements_B1_site_list)) for i in range(len(elements_B2_site_list))]
    #B_site_indexes = [i + (len(elements_A_site_list)) for i in range(len(elements_B_site_list))]

    if is_debug == True:
       print("A1_site_index:", A1_site_indexes)
       print("A2_site_index:", A2_site_indexes)
       print("B1_site_index:", B1_site_indexes)
       print("B2_site_index:", B2_site_indexes)

       print("X_init:", X_init)
       print("y_init:", y_init)

    properties_A = {}
    average_properties_A = {}
    properties_B = {}
    average_properties_B = {}

    #print("average properties:")
    #print("average_properties_A", average_properties_A)
    #print("average_properties_B", average_properties_B)
    
    # -----------------------------#
    # BEGIN: printing the features #
    # -----------------------------#

    division_value = 100.0

    AB_ratio = 0.0
    AB_product = 0.0
    for i in range(len(X_init)):

        properties_A1, average_properties_A1, properties_B1, average_properties_B1 = get_properties(is_debug=is_debug,
                                                         A_site_elements=elements_A1_site_list,
                                                         A_site_compositions=X_init[i][0:A1_site_indexes[-1] + 1],
                                                         B_site_elements=elements_B1_site_list,
                                                         B_site_compositions=X_init[i][B1_site_indexes[0]:B1_site_indexes[-1] + 1])

        properties_A2, average_properties_A2, properties_B2, average_properties_B2 = get_properties(is_debug=is_debug,
                                                         A_site_elements=elements_A2_site_list,
                                                         A_site_compositions=X_init[i][A1_site_indexes[-1] + 1:A2_site_indexes[-1] + 1],
                                                         B_site_elements=elements_B2_site_list,
                                                         B_site_compositions=X_init[i][B2_site_indexes[0]:])


        if is_print_unit_formula == True:
           print(
                  "(" + "".join(str(k) + str(l) for k, l in zip(elements_A1_site_list, X_init[i][0:A1_site_indexes[-1] + 1]) if k != 0 and l != 0) + ")"  + \
                  "(" + "".join(str(k) + str(l) for k, l in zip(elements_A2_site_list, X_init[i][A1_site_indexes[-1] + 1:A2_site_indexes[-1] + 1]) if k != 0 and l != 0) + ")"  + \
                  "(" + "".join(str(k) + str(l) for k, l in zip(elements_B1_site_list, X_init[i][A2_site_indexes[-1] + 1:B2_site_indexes[-1] + 1]) if k != 0 and l != 0) + ")"  + \
                  "(" + "".join(str(k) + str(l) for k, l in zip(elements_B2_site_list, X_init[i][B2_site_indexes[0]:]) if k != 0 and l != 0) + ")" + "X3",
                  end="            "
                 )


        for jProperties, kProperties in zip(properties_A1, properties_B1): # python is so great: 07/12/21

            AB_ratio = 0.0
            AB_product = 0.0

            AB_ratio = (average_properties_A1[jProperties]/average_properties_B1[kProperties])
            AB_product = (average_properties_A1[jProperties]/division_value)*(average_properties_B1[kProperties]/division_value)

            #print(average_properties_A[jProperties]/division_value, average_properties_B[kProperties]/division_value, end="  ")

            print(average_properties_A1[jProperties]/division_value, average_properties_B1[kProperties]/division_value, AB_ratio, AB_product, end="  ") # all

        #print("   ", y_init[i])

        for jProperties, kProperties in zip(properties_A2, properties_B2): # python is so great: 07/12/21

            AB_ratio = 0.0
            AB_product = 0.0

            #print("\n--------->>>>:", average_properties_A2[jProperties], average_properties_A2)
            #print("\n--------->>>>:", average_properties_B2[kProperties], average_properties_B2)

            if average_properties_A2[jProperties] == 0.0 or average_properties_B2[kProperties] == 0.0:

               AB_ratio = 0.0
               AB_product = 0.0

            else:

               AB_ratio = (average_properties_A2[jProperties]/average_properties_B2[kProperties])
               AB_product = (average_properties_A2[jProperties]/division_value)*(average_properties_B2[kProperties]/division_value)

            #print(average_properties_A[jProperties]/division_value, average_properties_B[kProperties]/division_value, end="  ")

            print(average_properties_A2[jProperties]/division_value, average_properties_B2[kProperties]/division_value, AB_ratio, AB_product, end="  ") # all

        print("   ", y_init[i])

    # --------------------------#
    # END printing the features #
    # --------------------------#

    #get_all_combination_of_tolerance_factors_of_AB(A_site_elements=elements_A_site_list,
    #                                               B_site_elements=elements_B_site_list)

In [22]:
%%capture cap --no-stderr
# To see output instead of saving it into a file in the next cell, uncomment the line above


if __name__ == "__main__":

##    case 01: energy_storagy.txt
#     elements_A1_site_list = ["Ba", "Ca", "Sr", "Cd", "Sm"]
#     elements_A2_site_list = ["Ca", "Sm"]
#     elements_B1_site_list = ["Al", "Fe", "Ti", "Zr", "Sn", "Hf", "Ni"]
#     elements_B2_site_list = ["Bi", "Ni", "Mn"]

# case 02: database/Johns_database.txt
    elements_A1_site_list = ['Ba', 'Bi', 'Ca', 'Dy', 'Eu',
                             'Gd', 'La', 'Li', 'Lu', 'Na',
                             'Pr', 'Sm', 'Sr', 'Tb', 'Y'] #15
    elements_A2_site_list = ['Ba', 'Ca', 'Cu', 'Eu', 'Gd',
                             'Mg', 'Na', 'Sm', 'Sr'] #9
    elements_B1_site_list = ['Ag', 'Al', 'Bi', 'Ce', 'Co', 
                             'Cr', 'Dy', 'Eu', 'Fe', 'Ga', 
                             'Gd', 'In', 'La', 'Mg', 'Mn',
                             'Nd', 'Ni', 'Pr', 'Sc', 'Sm',
                             'Ta', 'Ti', 'Zn'] #23
    elements_B2_site_list = ['Bi', 'Cr', 'Fe', 'I', 'Mn',
                             'Mo', 'Nb', 'Ta', 'Te', 'Ti',
                             'V', 'W'] # 12
# case 03: database/Johns_2.txt
    is_debug_true = False
    is_print_unit_formula = True

    main(is_debug_true,
        is_print_unit_formula,
        elements_A1_site_list,
        elements_A2_site_list,
        elements_B1_site_list,
        elements_B2_site_list,
        #file_name="energy_storagy.txt")
        file_name="database/Johns_database.txt")
        #file_name="database/Johns_database1.txt")

In [23]:
# Saving the output into output.csv file
with open('output.csv', 'w') as f:
    f.write(cap.stdout)